In [1]:
## Feature grouping by Supplier i.e. creating matrix of Supplier+Item-Features

In [1]:
import pandas as pd
import numpy as np
import scipy
from nltk.corpus import stopwords
import re
import os

In [2]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)

tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3] ## Remove unnecessary columns

tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case


tranzact_data3 = tranzact_data2.dropna() ## remove rows with cell value none

tranzact_data3 = tranzact_data3.reset_index(drop=True)

## Remove items sold by a company to its subsidiary :

drop_list = []

for i in range(len(tranzact_data3)):
    
    if (tranzact_data3['Supplier'][i].split())[0] == 'CESARE'and (tranzact_data3['Company'][i].split())[0] == 'CESARE':
        
        drop_list.append(i)
        
        tranzact_data = tranzact_data3.drop(drop_list)
                
tranzact_data = tranzact_data.reset_index(drop=True)

print(len(tranzact_data))
tranzact_data.head()       

24348


,Company,Supplier,Item
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
3,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
4,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."


In [3]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap

## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
words.append('min')
#words.append('machined')
words.append('top')
words.append('dia')
words.append('bar')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

print(len(tranzact_data))
print('================')
print(tranzact_data['features'][111])
print(type(tranzact_data['features'][111]))

## Removing duplicate words from individual features

## tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
##                                                                                key=x.split().index))]).lower())
print(len(tranzact_data))

## Removing only single word features :

tranzact_data = tranzact_data[tranzact_data['features'].str.contains(' ')]
tranzact_data = tranzact_data.reset_index(drop=True)

tranzact_data.tail()

24348
chromium vanadium spring steel rolled en47 wire rod
<class 'str'>
24348


,Company,Supplier,Item,features
22325,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
22326,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
22327,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
22328,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
22329,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [4]:
for i in range(len(tranzact_data)):
    if tranzact_data['features'][i] == '':
        
        print(i)
        print('===')
        tranzact_data['Supplier'][i] = np.nan
    
tranzact_data = tranzact_data.dropna()
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))
tranzact_data.tail()

22330


,Company,Supplier,Item,features
22325,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
22326,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
22327,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
22328,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
22329,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [5]:
## Removing duplicate features + Supplier combinations :

tranzact_data['find_duplicate'] = tranzact_data['Supplier'].map(str) + tranzact_data['features']
tranzact_data.drop_duplicates(subset='find_duplicate', keep = 'first', inplace = True)
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))
tranzact_data.tail()

16192


,Company,Supplier,Item,features,find_duplicate
16187,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11,M/S. S. K. Enterprisepvc gland pg11
16188,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13,M/S. S. K. Enterprisepvc gland pg13
16189,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1,M/S. S. K. Enterprisepvc gland m25x1
16190,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 1mm '1',ferrul number,M/S. S. K. Enterpriseferrul number
16191,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long,Anant Tradershard copper pipe gauge long


In [37]:
## Removing unnecessary long features ( feature words > 25):

tranzact_data['number_of_words'] = tranzact_data.features.apply(lambda x: len(x.split()))

list_to_drop = []

for i in range(len(tranzact_data['number_of_words'])):
    
    if tranzact_data['number_of_words'][i] > 50 :
        
        list_to_drop.append(i)

tranzact_data_final = tranzact_data.drop(list_to_drop)
tranzact_data_final = tranzact_data_final.reset_index(drop=True)


tranzact_data_final.tail()

,Company,Supplier,Item,features,find_duplicate,number_of_words
16177,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11,M/S. S. K. Enterprisepvc gland pg11,3
16178,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13,M/S. S. K. Enterprisepvc gland pg13,3
16179,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1,M/S. S. K. Enterprisepvc gland m25x1,3
16180,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 1mm '1',ferrul number,M/S. S. K. Enterpriseferrul number,2
16181,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long,Anant Tradershard copper pipe gauge long,5


In [42]:
index = [0,1,2,3,4,5,6, 7]
s = pd.Series(['m','n','o','p','q','r','v', 'u'],index= index)
t = pd.Series(['a','b','c','d','e','f','g', 'h'],index= index)
u = pd.Series(['A','B','A','B','B', 'C', 'A','B'],index=index)

df = pd.DataFrame({'MUL1':s,'MUL2':t,'MUL3':u})
df_new = df.copy()

print (df_new)


  MUL1 MUL2 MUL3
0    m    a    A
1    n    b    B
2    o    c    A
3    p    d    B
4    q    e    B
5    r    f    C
6    v    g    A
7    u    h    B


In [47]:

tranzact_data_final['supp_id'] = tranzact_data_final['Supplier'].factorize()[0]


tranzact_data_supplier = tranzact_data_final[['Supplier', 'supp_id']].drop_duplicates()

tranzact_data_supplier = tranzact_data_supplier.reset_index(drop=True)

tranzact_data_supplier.tail()
#print(df_new_1)


#tranzact_data_search['Supplier_id'] = tranzact_data_search['Supplier'].factorize()[0]  ## Indexing

#supplier_id_tranzact_data_search = tranzact_data_search[['Supplier', 'Supplier_id']].drop_duplicates(). \
#                                                        sort_values('Supplier_id')

,Supplier,supp_id
2246,S C Industrial Syndicate,2246
2247,Maxell Heat Exchangers Pvt Ltd,2247
2248,Imerys Minerals Malaysia Sdn Bhd (86796-M),2248
2249,Sigma Roto Lining LLP,2249
2250,Guangdong Yuxing Fire-retardant New Materials ...,2250


In [55]:
tranzact_data_final_copy = tranzact_data_final.copy()

tranzact_data_search = pd.DataFrame()

for i in tranzact_data_supplier['Supplier']:
    
    df1 = tranzact_data_final_copy.loc[tranzact_data_final_copy['Supplier'] == i, 'features'].reset_index()
    data1 = ''
    data2 = ''
    for j in range(len(df1)):
        if j == 0:
            data1 = data1 + df1['features'][j]
        else:
            data1 = data1 + ' ' + df1['features'][j]
    data2 = [[data1, i]]
    tranzact_data_search = tranzact_data_search.append(pd.DataFrame(data2, columns = ['Features', 'supplier']))
    tranzact_data_final_copy = tranzact_data_final_copy[tranzact_data_final_copy.Supplier != i]
tranzact_data_search = tranzact_data_search.reset_index()
tranzact_data_search.tail()


,index,Features,supplier
2246,0,5hp 2pole phase foot mounted ng132s cg motor,S C Industrial Syndicate
2247,0,dx coolingcoil wxh x32 x6r two compressor 13fp...,Maxell Heat Exchangers Pvt Ltd
2248,0,imercarb 1t,Imerys Minerals Malaysia Sdn Bhd (86796-M)
2249,0,coating etfe micron antistatic 2m anf,Sigma Roto Lining LLP
2250,0,yt ii,Guangdong Yuxing Fire-retardant New Materials ...


In [54]:
df_new = df.copy()
df2 = pd.DataFrame()
for i in df_new_1['MUL3']:
    
    df1 = df_new.loc[df_new['MUL3'] == i, 'MUL1'].reset_index()
    data1 = ''
    data2 = ''
    for j in range(len(df1)):
        if j == 0:
            data1 = data1 + df1['MUL1'][j]
        else:
            data1 = data1 + ' ' + df1['MUL1'][j]
    data2 = [[data1, i]]
    df2 = df2.append(pd.DataFrame(data2, columns = ['mul1', 'mul3']))
    df_new = df_new[df_new.MUL3 != i]
#    print(df_new)
df2 = df2.reset_index()
print(df2)
print('=============')
print(df2['mul1'])
print('=============')
print(df2['mul1'][0])


   index     mul1 mul3
0      0    m o v    A
1      0  n p q u    B
2      0        r    C
0      m o v
1    n p q u
2          r
Name: mul1, dtype: object
m o v


In [84]:
grouped = df.groupby('MUL3')['MUL2']
print(grouped)

# group.reset_index(drop=True, inplace=True)
for name, group in grouped:
    
    print(name)
    print(group)

print(type(grouped))
print(type(group))
print(type(name))
##  grouped = df.reset_index().groupby('A')            

A
0    a
2    c
5    f
Name: MUL2, dtype: object
B
1    b
3    d
4    e
6    g
Name: MUL2, dtype: object
<class 'pandas.core.groupby.groupby.SeriesGroupBy'>
<class 'pandas.core.series.Series'>
<class 'str'>


In [78]:
grouped = df.groupby('MUL3')['MUL2']

grouped = grouped.apply(lambda x: x.reset_index())
print(grouped)
print(type(grouped))

for name, group in grouped:
    print(name)
    
    print(len(group))
    print(type(group))
    
    x = ''
    for i in range(len(group)):
        x = x + ' ' + group[i]
    print(x)

        index    A    B
MUL3                   
A    0      0    a  NaN
     1      2    c  NaN
     2      5    f  NaN
B    0      1  NaN    b
     1      3  NaN    d
     2      4  NaN    e
     3      6  NaN    g
<class 'pandas.core.frame.DataFrame'>


ValueError: too many values to unpack (expected 2)

In [5]:
## To check no. of Suppliers & no. of unique featuresin the data base :

# print('Supplier = ', len(tranzact_data.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data.features.value_counts()))
# print('Tranzact_data length = ', len(tranzact_data))
# tranzact_data.tail()

In [7]:
## Above indicates that there are 2337 suppliers supplying 28921 items(as per no. of rows)
## above indicates that there are 16728 unique features out of 28921 items(as per rows)

In [8]:
## Let's reduce no. of Suppliers : criterian - apperaing at least 10 times in the data frame

# tranzact_data_short = tranzact_data.groupby("Supplier").filter(lambda x: len(x) > 10)

# print(len(tranzact_data_short))
# print(len(tranzact_data_short.Supplier.value_counts()))

In [9]:
## Above indicates that there are now 391 suppliers supplying 23649 items(as per no. of rows)

In [8]:
## To cross check no. of Suppliers & no. of unique features in the data base :

# print('Supplier = ', len(tranzact_data_final.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data_final.features.value_counts()))
# print('Total no. of entrys = ', len(tranzact_data_final))
# tranzact_data_final.tail()

In [56]:
# Creating seacrh engine file  and digitising original features :

from sklearn.feature_extraction.text import TfidfVectorizer


print(len(tranzact_data_search['Features']))
print(len(tranzact_data_search['Features'][0]))
print(tranzact_data_search['Features'][0])

my_vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', ngram_range=(1, 2))
digital_features = my_vectorizer.fit(tranzact_data_search['Features'])
my_vector = digital_features.transform(tranzact_data_search['Features']).toarray()
print(my_vector.shape)

tranzact_data_search.tail()


2251
30
zincoplast zinc zincoplast pyn
(2251, 56440)


,index,Features,supplier
2246,0,5hp 2pole phase foot mounted ng132s cg motor,S C Industrial Syndicate
2247,0,dx coolingcoil wxh x32 x6r two compressor 13fp...,Maxell Heat Exchangers Pvt Ltd
2248,0,imercarb 1t,Imerys Minerals Malaysia Sdn Bhd (86796-M)
2249,0,coating etfe micron antistatic 2m anf,Sigma Roto Lining LLP
2250,0,yt ii,Guangdong Yuxing Fire-retardant New Materials ...


In [72]:
## Input Text data :

text_data = ['pin uni1707']

#text_data_lower_case = [x.lower() for x in text_data]


#text_data1 = pd.Series(text_data_lower_case).apply(lambda x:sorted(set(x.split())))


print(text_data)
print(type(text_data))

['pin uni1707']
<class 'list'>


In [73]:
## From Teaxt data to text feature :

text_data_lower_case = [x.lower() for x in text_data]

text_data1 = pd.Series(text_data_lower_case)
#print(type(text_data1))
#print('==================')

#text_data1 = pd.Series(text_data1.apply(lambda x:sorted(set(x.split()))))
#print(type(text_data1))

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
#words.append('machined')
words.append('top')
words.append('dia')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

# text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
#                                                                                key=x.split().index))]).lower())

print(text_data_feature)
print("=========")
print(type(text_data_feature))
print('==================')
print(len(tranzact_data_search['Features']))
## Cosine Similarity :

def cos_cdist(matrix, vector):
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

## Creating test vector from Text features :

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []


merge_text_feature = tranzact_data_search['Features'].append(pd.Series(text_data_feature))


print('==================')
print(len(tranzact_data_search['Features']))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()

test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

print('%%%%%%%%%%%%%%%%%%%%%%%%%%')
print(merge_text_feature.shape)
print('===================')
print(merge_digit_vector.shape)
print('===================')
print(type(my_vector))
print(my_vector.shape)
print('===================')
print(type(test_digit_vector))
print(test_digit_vector.shape)
print('%%%%%%%%%%%%%%%%%%%%%%%%%')


## Finding cosine similarity :

c_d = []

# c_d = np.round((1 - cos_cdist(my_vector, test_digit_vector)), 3)
c_d = np.round((1 - cos_cdist(merge_digit_vector[0:len(merge_digit_vector)-2], test_digit_vector)), 3)
print(type(c_d))
print(c_d.shape)
print(text_data_feature)
print('+++++++++++')
print(len(tranzact_data_search))
#merge_text_feature.tail()

0    pin uni1707
dtype: object
<class 'pandas.core.series.Series'>
2251
2251
%%%%%%%%%%%%%%%%%%%%%%%%%%
(2252,)
(2252, 56440)
<class 'numpy.ndarray'>
(2251, 56440)
<class 'numpy.ndarray'>
(56440,)
%%%%%%%%%%%%%%%%%%%%%%%%%
<class 'numpy.ndarray'>
(2250,)
0    pin uni1707
dtype: object
+++++++++++
2251


In [74]:
result = []
result_frame = []
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-50:][::-1]
#max_c_s_index = np.argsort(aa_sum)[-50:][::-1]

for i in max_c_s_index:
    
      
    result.append(np.array([c_d[i], tranzact_data_search['Features'][i], \
                            tranzact_data_search['supplier'][i]]))

        
    
#    result.append(np.array([aa_sum[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
#                            tranzact_data_search['Supplier'][i]]))


result_frame = pd.DataFrame(result, columns=['c_index', 'Features', 'supplier'])


result_frame.c_index = result_frame.c_index.astype('float64')

#result_frame.drop_duplicates(subset='supplier', keep = 'first', inplace = True)
#result_frame = result_frame.reset_index(drop=True)

kount = 0

for i in range(len(result_frame['supplier'])):
    if result_frame['c_index'][i] >= 0.10:
        print(result_frame['supplier'][i])
        print(i)
        kount = kount+1
        if kount == 5 :
            break
result_frame.head(10)

HAFSA INDUSTRIES
0
Radhe Engineering
1
DAMODAR ENTERPRISES
2
VEER TOOLS
3
Maa Sitola Engineering Works
4


,c_index,Features,supplier
0,0.251,segmental ring gtv jam nut ps gasket segmental...,HAFSA INDUSTRIES
1,0.148,stopper pin,Radhe Engineering
2,0.126,10x hand magnifier ken5186000k 5x hand magnifi...,DAMODAR ENTERPRISES
3,0.122,spring dewal pin split dowel pin split pin,VEER TOOLS
4,0.102,shaft tpw pin tpw shaft pin tpw,Maa Sitola Engineering Works
5,0.090,2nf 2200pf 250vac x1 y1 pin 33uf 400v pin,MAULIK ELECTRO SALES
6,0.087,pin connector male 5sqmm ip65 pin connector fe...,Opulence Engineering Solutions
7,0.087,lm3914 pin dip ttype,LIBRAN COMPONENTS
8,0.085,panel door part siemens type bend pin siemens ...,N-Rack Accessories
9,0.081,slide switch angled tactile switch pin ap 05h ...,SWITCH POINT


In [62]:
print(result_frame['Features'][1])
print(result_frame['Features'][2])

2way npt ss316 body nc orifice opr pressure bar weather proof enclosure ip67 manual override class coil insulation
electro pneumatic smart ff positioner make fisher model dvc6200f foundation fieldbus weather proof electrical connection ip65 pneumatic connection


In [75]:
for j in range(10):
    
    if result_frame['Features'][j] != result_frame['Features'][j+1]:
        
                
        next_text_data = [result_frame['Features'][j] + ' ' + result_frame['Features'][j+1]]
                                        
        break
print(next_text_data)
print(type(next_text_data))

['segmental ring gtv jam nut ps gasket segmental ring gland flange ggv pin d12x pin d12 30l support ring scv reducer atex 4npt 2npt zn plt cs thrust plate gland flange double handle htb carbon steel support ring dowel pin uni1707 d16 l90 dowel pin uni1707 d12 l60 pin l25 segmental ring gtv aisi420 bracket for ge namaplate gland bolt x10unc l79 a193 b8 stopper pin']
<class 'list'>


In [76]:
## From Teaxt data to text feature :

text_data_lower_case = [x.lower() for x in next_text_data]

text_data1 = pd.Series(text_data_lower_case)
#print(type(text_data1))
#print('==================')

#text_data1 = pd.Series(text_data1.apply(lambda x:sorted(set(x.split()))))
#print(type(text_data1))

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
#words.append('machined')
words.append('top')
words.append('dia')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

# text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
#                                                                                key=x.split().index))]).lower())

print(text_data_feature)
print("=========")
print(type(text_data_feature))
print('==================')
print(len(tranzact_data_search['Features']))
## Cosine Similarity :

def cos_cdist(matrix, vector):
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

## Creating test vector from Text features :

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []


merge_text_feature = tranzact_data_search['Features'].append(pd.Series(text_data_feature))


print('==================')
print(len(tranzact_data_search['Features']))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()

test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

print('%%%%%%%%%%%%%%%%%%%%%%%%%%')
print(merge_text_feature.shape)
print('===================')
print(merge_digit_vector.shape)
print('===================')
print(type(my_vector))
print(my_vector.shape)
print('===================')
print(type(test_digit_vector))
print(test_digit_vector.shape)
print('%%%%%%%%%%%%%%%%%%%%%%%%%')


## Finding cosine similarity :

c_d = []

# c_d = np.round((1 - cos_cdist(my_vector, test_digit_vector)), 3)
c_d = np.round((1 - cos_cdist(merge_digit_vector[0:len(merge_digit_vector)-2], test_digit_vector)), 3)
print(type(c_d))
print(c_d.shape)
print(text_data_feature)
print('+++++++++++')
print(len(tranzact_data_search))
#merge_text_feature.tail()

0    segmental ring gtv jam nut ps gasket segmental...
dtype: object
<class 'pandas.core.series.Series'>
2251
2251
%%%%%%%%%%%%%%%%%%%%%%%%%%
(2252,)
(2252, 56441)
<class 'numpy.ndarray'>
(2251, 56440)
<class 'numpy.ndarray'>
(56441,)
%%%%%%%%%%%%%%%%%%%%%%%%%
<class 'numpy.ndarray'>
(2250,)
0    segmental ring gtv jam nut ps gasket segmental...
dtype: object
+++++++++++
2251


In [77]:
result = []
result_frame = []
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-50:][::-1]
#max_c_s_index = np.argsort(aa_sum)[-50:][::-1]

for i in max_c_s_index:
    
      
    result.append(np.array([c_d[i], tranzact_data_search['Features'][i], \
                            tranzact_data_search['supplier'][i]]))

        
    
#    result.append(np.array([aa_sum[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
#                            tranzact_data_search['Supplier'][i]]))


result_frame = pd.DataFrame(result, columns=['c_index', 'Features', 'supplier'])


result_frame.c_index = result_frame.c_index.astype('float64')

#result_frame.drop_duplicates(subset='Supplier', keep = 'first', inplace = True)
#result_frame = result_frame.reset_index(drop=True)

kount = 0

for i in range(len(result_frame['supplier'])):
    if result_frame['c_index'][i] >= 0.10:
        print(result_frame['supplier'][i])
        print(i)
        kount = kount+1
        if kount == 5 :
            break
result_frame.head(10)

HAFSA INDUSTRIES
0
SPA Engineering
1
Radhe Engineering
2
VAKRATUNDA
3
Tirupati Enterpries
4


,c_index,Features,supplier
0,0.987,segmental ring gtv jam nut ps gasket segmental...,HAFSA INDUSTRIES
1,0.281,segmental ring segmental ring gtv ps gasket di...,SPA Engineering
2,0.171,stopper pin,Radhe Engineering
3,0.160,lock nut gland flange,VAKRATUNDA
4,0.140,lever nut anti ext ring dn100 anti ext ring dn...,Tirupati Enterpries
5,0.111,gland flange ggv gland flange gland flange for...,RK Enterprises
6,0.109,plug npt cushion ring for bc1 glass support ri...,Dwekam Precision Industries
7,0.108,carbon ring gland paking stem,ACE CARBON
8,0.108,spring dewal pin split dowel pin split pin,VEER TOOLS
9,0.107,rlg water column assm for probes segmental rin...,AKASH INDUSTRIES


In [78]:
print(result_frame['Features'][1])

segmental ring segmental ring gtv ps gasket disc nut disc nut 900tcv pr seal gasket 1500tcv support ring segmental ring glv segmental ring scv
